In [1]:
!pip install konlpy
!pip install tqdm

     |████████████████████████████████| 19.4 MB 8.0 MB/s 
     |████████████████████████████████| 448 kB 35.4 MB/s 


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
from gensim.models.word2vec import Word2Vec
from konlpy.tag import Okt
from tqdm import tqdm

In [3]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")

('ratings.txt', <http.client.HTTPMessage at 0x7f97878da610>)

In [4]:
train_data = pd.read_table('ratings.txt')

In [5]:
train_data[:5] # 상위 5개 출력

id                                           document  label
0   8112052                                어릴때보고 지금다시봐도 재밌어요ㅋㅋ      1
1   8132799  디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...      1
2   4655635               폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.      1
3   9251303  와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...      1
4  10067386                        안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.      1

In [6]:
print(len(train_data)) # 리뷰 개수 출력

200000


In [7]:
print(train_data.isnull().values.any())

True


In [8]:
train_data = train_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(train_data.isnull().values.any()) # Null 값이 존재하는지 확인


False


In [9]:
print(len(train_data)) # 리뷰 개수 출력

199992


In [10]:
# 정규 표현식을 통한 한글 외 문자 제거
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [11]:
train_data[:5] # 상위 5개 출력

id                                           document  label
0   8112052                                어릴때보고 지금다시봐도 재밌어요ㅋㅋ      1
1   8132799  디자인을 배우는 학생으로 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업...      1
2   4655635                   폴리스스토리 시리즈는 부터 뉴까지 버릴께 하나도 없음 최고      1
3   9251303   와 연기가 진짜 개쩔구나 지루할거라고 생각했는데 몰입해서 봤다 그래 이런게 진짜 영화지      1
4  10067386                         안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화      1

In [13]:
# 불용어 정의
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

# 형태소 분석기 OKT를 사용한 토큰화 작업 (다소 시간 소요)
okt = Okt()

tokenized_data = []
train_data = train_data[0:10000]
for sentence in tqdm(train_data['document']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    tokenized_data.append(stopwords_removed_sentence)

100%|██████████| 10000/10000 [00:54<00:00, 182.77it/s]


In [14]:
len(tokenized_data)

10000

In [15]:
from gensim.models import Word2Vec

model = Word2Vec(sentences = tokenized_data, size = 100, window = 5, min_count = 5, workers = 4, sg = 0)

In [16]:
# 완성된 임베딩 매트릭스의 크기 확인
model.wv.vectors.shape

(2448, 100)

In [17]:
model.wv.vocab

{'어리다': <gensim.models.keyedvectors.Vocab at 0x7f9763bf5dd0>,
 '때': <gensim.models.keyedvectors.Vocab at 0x7f9763bf5e50>,
 '보고': <gensim.models.keyedvectors.Vocab at 0x7f9763bf5a90>,
 '지금': <gensim.models.keyedvectors.Vocab at 0x7f9763bf57d0>,
 '다시': <gensim.models.keyedvectors.Vocab at 0x7f9763bf5810>,
 '보다': <gensim.models.keyedvectors.Vocab at 0x7f976318b450>,
 '재밌다': <gensim.models.keyedvectors.Vocab at 0x7f976318b490>,
 'ㅋㅋ': <gensim.models.keyedvectors.Vocab at 0x7f976318b4d0>,
 '을': <gensim.models.keyedvectors.Vocab at 0x7f9763bf5ed0>,
 '배우다': <gensim.models.keyedvectors.Vocab at 0x7f976322d250>,
 '학생': <gensim.models.keyedvectors.Vocab at 0x7f976318b510>,
 '외국': <gensim.models.keyedvectors.Vocab at 0x7f976318b550>,
 '그': <gensim.models.keyedvectors.Vocab at 0x7f976318b590>,
 '전통': <gensim.models.keyedvectors.Vocab at 0x7f976318b5d0>,
 '통해': <gensim.models.keyedvectors.Vocab at 0x7f976318b610>,
 '발전': <gensim.models.keyedvectors.Vocab at 0x7f976318b650>,
 '문화': <gensim.models.ke

In [18]:
model.wv.vectors

array([[ 0.56866586,  0.14388832, -0.5489022 , ..., -0.15906173,
         0.14915766, -0.5439707 ],
       [ 0.6003524 ,  0.16506925, -0.53920263, ..., -0.14424428,
         0.11921345, -0.5039577 ],
       [ 0.6366254 ,  0.18412852, -0.5884054 , ..., -0.17482965,
         0.16157593, -0.5860948 ],
       ...,
       [ 0.14341141,  0.03709248, -0.12658785, ..., -0.04628074,
         0.03095972, -0.13398989],
       [ 0.10248662,  0.02388121, -0.08937459, ..., -0.02492923,
         0.02937449, -0.09378559],
       [ 0.10057054,  0.03085374, -0.08900718, ..., -0.02503764,
         0.02454063, -0.08627138]], dtype=float32)

In [19]:
print(model.wv.most_similar("감독"))

[('대한', 0.9998388886451721), ('버리다', 0.9998281598091125), ('그녀', 0.9998226761817932), ('그리고', 0.9998225569725037), ('걸작', 0.9998185634613037), ('내다', 0.9998183846473694), ('그렇다', 0.9998180866241455), ('전쟁', 0.9998149275779724), ('속', 0.9998118877410889), ('크다', 0.9998088479042053)]


In [20]:
print(model.wv.most_similar("멋지다"))

[('급', 0.9999018907546997), ('완벽하다', 0.9998840689659119), ('다', 0.9998816251754761), ('내다', 0.9998793601989746), ('크다', 0.9998791217803955), ('이라', 0.9998788237571716), ('끄다', 0.9998783469200134), ('코미디', 0.9998778104782104), ('아주', 0.9998741149902344), ('시키다', 0.9998728036880493)]


- 구글의 사전훈련된 w2v 모형
- https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit
- 300만 문헌*300개 임베딩